# Kaggle API setup to upload data set.

In [ ]:
# installing kaggle
!pip install -q kaggle

# Upload your kaggle api key

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import os
os.makedirs("/content/cecs456_project", exist_ok=True)
%cd /content/cecs456_project

In [ ]:
# downloads zip file from kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
# unzip zip file with images into folder
!unzip chest-xray-pneumonia.zip

# Possible Issue and fix for duplicating files
** Note ** <br>
When unziping files, there is a possibility that some of the files will duplicate. If that occurs, run these command(s) (which ever applies)<br>
!rm -rf /content/cecs456_project/chest_xray/__MACOSX <br>
!rm -rf /content/cecs456_project/chest_xray/chest_xray <br>
in the following cells to remove them.

In [ ]:
# removes meta folder
!rm -rf /content/cecs456_project/chest_xray/__MACOSX
print("_MACOSX folder removed")

In [ ]:
# removes nested duplicate chest_xray folder
!rm -rf /content/cecs456_project/chest_xray/chest_xray
print("Nested chest_xray folder removed")

# Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
np.random.seed(42)
import matplotlib.pyplot as plt
tf.random.set_seed(42)

# Loading the Dataset

In [ ]:
# setting up the paths to the datasets
training_set = os.path.join('/content/cecs456_project/chest_xray', 'train')
testing_set = os.path.join('/content/cecs456_project/chest_xray', 'test')

# resizes images to a lower scale
IMG_HEIGHT = 180
IMG_WIDTH = 180
# number of images processed at a time
BATCH_SIZE = 16
# uses 20% of the training data for validation
VALIDATION_SPLIT = 0.2
# ensures reproducability of training and validation splits
SEED = 42

In [ ]:
# scans folder and counts files to store file path
# creating file paths and instructions to how to resize and preprocess the images
# the traing will take 80% of the training set, validating will take the other 20%, and testing will test from the testing set

# training pathway
training_images = tf.keras.preprocessing.image_dataset_from_directory(training_set, validation_split = VALIDATION_SPLIT, subset = "training", seed = SEED, image_size = (IMG_HEIGHT, IMG_WIDTH), batch_size = BATCH_SIZE, label_mode = "binary", shuffle = True)
# validating pathway
validating_images = tf.keras.preprocessing.image_dataset_from_directory(training_set, validation_split = VALIDATION_SPLIT, subset = "validation", seed = SEED, image_size = (IMG_HEIGHT, IMG_WIDTH), batch_size = BATCH_SIZE, label_mode = "binary", shuffle = True)
# testing pathway
testing_images = tf.keras.preprocessing.image_dataset_from_directory(testing_set, image_size = (IMG_HEIGHT, IMG_WIDTH), batch_size = BATCH_SIZE, label_mode = "binary", shuffle = False)


In [ ]:
print(f"Training will be done in {tf.data.experimental.cardinality(training_images).numpy()} batches of {BATCH_SIZE} images.")
print(f"Validation will be done in {tf.data.experimental.cardinality(validating_images).numpy()} batches of {BATCH_SIZE} images.")
print(f"Testing will be done in {tf.data.experimental.cardinality(testing_images).numpy()} batches of {BATCH_SIZE} images.")

# Building the CNN

In [ ]:
# initializes model
cnn_model = keras.Sequential(name = "cnn_model")

# rescales pixels to [0,1]
cnn_model.add(layers.Rescaling(1.0/255, input_shape =(IMG_HEIGHT, IMG_WIDTH, 3)))

# implements 4 convolutional blocks with 2 layers each

# first block for feature extraction with 32 filters
cnn_model.add(layers.Conv2D(32, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization()) #
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.Conv2D(32, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.MaxPool2D(2))

# second block with 64 filters
cnn_model.add(layers.Conv2D(64, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.Conv2D(64, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.MaxPool2D(2))

# third block with 128
cnn_model.add(layers.Conv2D(128, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.Conv2D(128, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.MaxPool2D(2))

# fourth block with 256
cnn_model.add(layers.Conv2D(256, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.Conv2D(256, 3, padding = "same", use_bias = False))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation('relu'))
cnn_model.add(layers.MaxPool2D(2))

# features flatten into 1D vector for classification
cnn_model.add(layers.Flatten())

# FC layer 1
cnn_model.add(layers.Dense(128, use_bias=False)) # changes from 128
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.Dropout(0.3))

# FC layer 2
cnn_model.add(layers.Dense(32, use_bias=False)) # changed from 32
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.Dropout(0.3))

# output layer
cnn_model.add(layers.Dense(1, activation = "sigmoid"))


In [ ]:
# displays model summary
cnn_model.summary()

# Compiling the CNN model

In [ ]:
# compiles model
# sets up the optimizer and loss function

cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
print("Model compiled")

# Training the CNN Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import time # to use timer to see how long model takes

print("Training has started")
print(f"Total Epochs: 20")
print(f"Batches size: {BATCH_SIZE}")
#
print(f"Training batches: {tf.data.experimental.cardinality(training_images).numpy()}")
print(f"Validation batches: {tf.data.experimental.cardinality(validating_images).numpy()}")
#
start_time = time.time() # timer begins

# implementing early stopping to prevent overfitting.
# early stopping will happen if there is no improvement after n epochs (implemented)
stop_early = EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True, verbose = 1)
history = cnn_model.fit(training_images, validation_data = validating_images, epochs = 20, callbacks= [stop_early] , verbose = 1)

# without implementing early stopping (not implemented)
#history = cnn_model.fit(training_images, validation_data = validating_images, epochs = 20, verbose = 1)

end_time = time.time() - start_time # total training time

print("Done!")
print(f"Total training time was: {int(end_time//60)} mins and {int(end_time%60)} seconds.")



# Training and Validation Graphs

In [ ]:
#
accuracy = history.history["accuracy"]
validation_accuracy = history.history['val_accuracy']
loss = history.history['loss']
validation_loss = history.history['val_loss']
epochs = range(1, len(accuracy) +1)

# ploting accuracy
plt.figure(figsize = (8,4))
plt.plot(epochs, accuracy, label = "Training Accuracy", color = "purple")
plt.plot(epochs, validation_accuracy, label = "Validation Accuracy", color = "green")
plt.title("Accuracy Graph")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show() # prints accuracy graph
print(f"\nFinal: \nTraining Accuracy: {accuracy[-1]:.3f} \nValidation Accuracy: {validation_accuracy[-1]:.3f}\n")

# plotting loss
plt.figure(figsize = (8,4))
plt.plot(epochs, loss, label = "Training Loss", color = "purple")
plt.plot(epochs, validation_loss, label = "Validation Loss", color = "green")
plt.title("Loss Graph")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()
print(f"\nFinal: \nTraining Loss: {loss[-1]:.3f} \nValidation Loss: {validation_loss[-1]:.3f}\n")


# Evaluation

In [ ]:
testing_loss, testing_accuracy = cnn_model.evaluate(testing_images, verbose = 1)
print(f"Total loss on Testing Set {testing_loss:.4f}")
print(f"Accuracy of Testing Set {testing_accuracy:.4f}")

# Predictions on the first 9 images

In [ ]:
# grabs the first 9 images
for xray_images, image_labels in testing_images.take(1):
  first9_imgs = xray_images[:9]
  first9_labls = image_labels[:9]
  break

class_name = ["NORMAL", "PNEUMONIA"]

print("Prediction Probabilites Results of the first 9 images")
# gets and prints the prediction probabilities
prediction_prob = cnn_model.predict(first9_imgs).flatten()
print(prediction_prob)

print("\nPrediction Results of the first 9 images")
# predicts and prints the class type (label)
prediction_class_type = (prediction_prob > 0.5).astype(int).flatten()
print(np.array(class_name)[prediction_class_type])

print("\nReal Labels")
# prints the real class type (label)
print(np.array(class_name)[first9_labls.numpy().astype(int).flatten()])

# Confusion Matrix and Classificaiton



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# grabs the predictions of the xray images
predicted_probabilites_of_xrays = cnn_model.predict(testing_images)
predicted_classes_of_xrays = (predicted_probabilites_of_xrays > 0.5).astype(int).flatten()
# grabs the actual real labels from the testing folder set
real_classes = np.concatenate([y for x, y in testing_images], axis = 0).astype(int)

print(f"Total number of Images: {len(real_classes)}")
print(f"Total NORMAL Images: {np.sum(real_classes == 0)}")
print(f"Total PNUEMONIA Images: {np.sum(real_classes == 1)}")

# Confusion Matrix
confus_matrix = confusion_matrix(real_classes, predicted_classes_of_xrays)

print("\n Confusion Matrix")
print("                      Predictions\n")
print("                    NORMAL PNEUMONIA")
print("                  ------------------")
print(f"           NORMAL |{confus_matrix[0,0]:4d}  | {confus_matrix[0,1]:4d}")
print("Actual            ------------------")
print(f"        PNEUMONIA |{confus_matrix[1,0]:4d}  | {confus_matrix[1,1]:4d}")
# Classificatin report. shows precision, recall, f1 score
print(f"\n\n{classification_report(real_classes, predicted_classes_of_xrays, target_names = ["NORMAL", "PNEUMONIA"])}")

# Visual Model Predictions

In [ ]:
# visual comparison of predictions vs actual
# displays first 9 images and labels, true and predicion w/ confident %
# the closer the prediction is to 0.0, the more confident the model thinks its NORMAL, closer to 1.0, it thinks its PNEUMONIA
# label will be green if model predicted correct, else it will be red

# plots the first 9 images
plt.figure(figsize = (6.4, 5.6))
for i in range(9):
  plt.subplot(3, 3, i+1)
  plt.imshow(first9_imgs[i].numpy().astype(int))
  plt.axis("off")

  # predicted and real labels
  real_labels = class_name[int(first9_labls[i])]
  predicted_labels = class_name[prediction_class_type[i]]
  probability = prediction_prob[i]

  # checks if prediction and actual matach and assigns green or red to color
  # green if correct, else red
  if prediction_class_type[i] == first9_labls[i]:
    color = "green"
  else:
    color = "red"
  plt.title(f"True Label: {real_labels}\n Prediction: {predicted_labels} ({probability:.2f})", fontsize = 10, color = color)

plt.tight_layout()
plt.show()

In [ ]:
# only run if you want to save the model
# saves trainig model for reproducibility
cnn_model.save("chest_xray_cnn_model.keras")


# Final Summary

In [ ]:
print("Final Summary Results\n")
print(f"Test Accuracy:    {testing_accuracy:.1%}")
print(f"Test Loss:        {testing_loss:.3f}")
print(f"Total Parameters: {cnn_model.count_params():,}")